#### <i> PODSUMOWANIE: 
    Zrealizowano całe zadanie.

## 1. Funkcja generowana, która oblicza średnią harmoniczną

In [1]:
# https://docs.julialang.org/en/release-0.4/manual/metaprogramming/#generated-functions
@generated function harmonic_mean{T,N}(values :: Vararg{T,N})
    res = foldl((acc, i) -> :($acc + (1/values[$i])), :(0), 1:N)
    return :(N/$res)
end

harmonic_mean (generic function with 1 method)

In [2]:
harmonic_mean(1,2,3,5)

1.9672131147540985

[Wolfram](http://www.wolframalpha.com/input/?i=harmonic+mean+1,2,3,5)


In [3]:
harmonic_mean(-1.0, 8.0, 42.0, 15.0)

-5.098634294385432

[Wolfram](http://www.wolframalpha.com/input/?i=harmonic+mean+-1.0,+8.0,+42,+15)

## 2. Algorytm automatycznego różniczkowania

#### - pochodna ze stałej (n/dx)

In [4]:
autodiff(n :: Number) = 0

autodiff (generic function with 1 method)

#### - pochodna ze zmiennej (x/dx)

In [5]:
autodiff(s :: Symbol) = 1

autodiff (generic function with 2 methods)

#### - pochodna sumy

In [6]:
function autodiff(::Type{Val{:+}}, ex...) :: Expr
    Expr(:call, :+, map(y -> autodiff(y), ex[1:end])...)
end

autodiff (generic function with 3 methods)

#### - pochodna różnicy

In [7]:
function autodiff(::Type{Val{:-}}, ex...) :: Expr
    Expr(:call, :-, autodiff(ex[1]), autodiff(ex[2]))
end

autodiff (generic function with 4 methods)

#### - pochodna iloczynu

In [8]:
function autodiff(::Type{Val{:*}}, ex...) :: Expr
    if (length(ex) == 2)
        a = ex[1] 
    else 
        a = Expr(:call, :*, ex[1:end-1]...)
    end
    
    b = ex[end]
    Expr(:call, :+, Expr(:call, :*, b, autodiff(a)), Expr(:call, :*, autodiff(b), a))
end

autodiff (generic function with 5 methods)

#### - pochodna ilorazu

In [9]:
function autodiff(::Type{Val{:/}}, ex...) :: Expr
    a = ex[1]
    b = ex[2]
    Expr(:call, :/,
        Expr(:call, :-, Expr(:call, :*, autodiff(a), b), Expr(:call, :*, a, autodiff(b))), Expr(:call, :^, b, 2))
end

autodiff (generic function with 6 methods)

#### - metoda "top level"

In [10]:
function autodiff(ex::Expr)::Expr
    autodiff(Val{ex.args[1]}, ex.args[2:end]...) 
end

autodiff (generic function with 7 methods)

In [11]:
dump(:(x + x*x*x))

Expr
  head: Symbol call
  args: Array{Any}((3,))
    1: Symbol +
    2: Symbol x
    3: Expr
      head: Symbol call
      args: Array{Any}((4,))
        1: Symbol *
        2: Symbol x
        3: Symbol x
        4: Symbol x
      typ: Any
  typ: Any


In [12]:
autodiff(:(x + x*x*x))

:(1 + (x * (x * 1 + 1x) + 1 * (x * x)))

In [13]:
autodiff(:(y + y*y*y - 7 * y))

:((1 + (y * (y * 1 + 1y) + 1 * (y * y))) - (y * 0 + 1 * 7))